## 3D Variable Surface Extraction
This notebook provides a specialized tool for extracting 2D "surface" data from 3D PALM (Potsdam Atmospheric Large-Eddy Simulation Model) NetCDF output files. Specifically, it extracts data from the lowest non-zero atmospheric layer and the layer immediately above it for predefined groups of 3D variables (e.g., wind components). The extracted 2D data is then saved into new, smaller NetCDF files,

## 1. Import Dependencies
This section imports all necessary Python libraries for numerical operations, NetCDF file handling, plotting (though not directly used for output here, good practice), and basic operating system interactions.

In [2]:
import os

import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt

## 2. Load 3D Simulation Data
This section defines the file paths for the 3D simulation output NetCDF files (for a baseline and a scenario run) and the static driver file. These files are then loaded into netCDF4 Dataset objects, providing access to the raw simulation data. The buildings_2d data from the static driver is also extracted, which might be useful for masking or contextualizing the extracted surface data.

In [3]:
# Absolute URLs (paths) of 3D simulation output files
file_3d_1 = r"./Data/_simulation_outputs_3/konstanz_4096x4096_v9_Baseline-48hr/OUTPUT/konstanz_4096x4096_v9_Baseline_av_3d_N03.000.nc"
file_3d_2 = r"./Data/_simulation_outputs_3/konstanz_4096x4096_v9_Scenario_1-48hr/OUTPUT/konstanz_4096x4096_v9_Scenario_1_av_3d_N03.000.nc"
file_static = r"./Data/_simulation_outputs_3/konstanz_4096x4096_v9_Scenario_1-48hr/INPUT/konstanz_4096x4096_v9_Scenario_1_static_N03"

# Read NetCDF files into Dataset objects in read mode ('r')
dataset_1 = nc.Dataset(file_3d_1, mode='r')
dataset_2 = nc.Dataset(file_3d_2, mode='r')
dataset_3 = nc.Dataset(file_static, mode='r')

# Store the 3D dataset objects in a list for easy iteration during processing.
file_xy_list = [file_3d_1, file_3d_2]
dataset_list = [dataset_1, dataset_2]

# Extract 2D building data from the static driver.
# This data might be used for masking non-atmospheric grid cells in the extracted surface data.
buildings_2d = dataset_3['buildings_2d']
buildings_2d_data = buildings_2d[:,:]

## 3. Define Time Sequences
This section dynamically extracts the total number of time steps from the loaded dataset. It then defines parameters for time step intervals and calculates time_sequence and time_sequence_all, which are used to control which time steps are processed and included in the output.

In [4]:
# Dynamically get the total number of time steps from the 'time' dimension of the first dataset.
total_time_steps = len(dataset_1.dimensions['time'])

# Define `time_step_interval`: This controls how frequently data points are sampled from the full time series.
# For example, if original data is every 10 min:
#   - `time_step_interval = 1`: Exports every 10 min data point.
#   - `time_step_interval = 6`: Exports every 6th data point, representing hourly data.
time_step_interval = 6

# Define `second_step`: This offset ensures that `time_sequence` starts at a meaningful interval if `time_step_interval` > 1.
# - For `time_step_interval = 6` (hourly data), `second_step = 5` means it selects time steps corresponding to 10, 20, ..., 50, 60 minutes.
#   Since time steps are 0-indexed and represent 10-minute intervals, step 5 corresponds to 0:50, step 11 to 1:50, etc.
#   If the intention is to grab the hour mark (e.g., 0:00, 1:00, 2:00), adjust `second_step` based on how `time_sequence` is generated.
if time_step_interval == 3:
    second_step = 2
elif time_step_interval == 6:
    second_step = 5
elif time_step_interval == 1:
    second_step = 0 # Start from the first time step (index 0)

# `time_sequence`: A list of selected time step indices, starting at `0` and extending by `time_step_interval`.
# This is typically used to extract a subset of the full time series.
time_sequence = [0]
time_sequence.extend(np.arange(second_step, total_time_steps, time_step_interval))

# `time_sequence_all`: A list containing all available time step indices (from 0 to `total_time_steps - 1`).
# This can be used if all time steps need to be processed/exported.
time_sequence_all = range(total_time_steps)

## 4. Define Variable Groups and Helper Functions
This section identifies all 3D variables in the dataset and categorizes them into predefined groups. It also includes helper functions to filter these groups for available variables and to extract unique dimensions for each group, which is crucial for structuring the output NetCDF files.

In [5]:
# Get all variable names available in the first dataset.
variable_names = dataset_1.variables.keys()
# Filter `variable_names` to include only those with more than 2 dimensions (typically 3D or 4D variables in PALM).
variable_names_palm = [var for var in variable_names if dataset_1.variables[var].ndim > 2]

print(f"Available 3D variables in dataset: {variable_names_palm}\n")

# Define predefined groups of variables. This allows for exporting related variables together.
# `variable_group_1`: Typically contains wind speed and direction.
variable_group_1 = ['wdir', 'wspeed']
# `variable_group_2`: Typically contains Cartesian wind components.
variable_group_2 = ['u', 'v', 'w']

# Filter each predefined variable group to ensure only variables actually present in the dataset are included.
# This prevents errors if a variable in the group is not in the loaded NetCDF file.
for variable_group in (variable_group_1, variable_group_2):
    updated_variable_group = []
    for variable in variable_group:
        if variable in variable_names_palm: # Check if the variable exists in the dataset's 3D variables.
            updated_variable_group.append(variable)
    variable_group.clear()  # Clear the original list.
    variable_group.extend(updated_variable_group)  # Update the list with the filtered variables.

def get_unique_dimensions(variable_group):
    """
    Retrieves the unique dimensions (e.g., 'time', 'z', 'y', 'x') present across all
    variables within a specified group. This helps in defining the structure of
    the output NetCDF file for that group.

    Args:
        variable_group (list): A list of variable names (strings).

    Returns:
        list: A list of unique dimension names (strings) for the variables in the group.
    """
    unique_dimensions = set() # Use a set to automatically handle unique dimensions.
    for variable_name_export in variable_group:
        try:
            # Get the dimensions tuple for the current variable.
            variable_dimensions = dataset_1.variables[variable_name_export].dimensions
            unique_dimensions.update(variable_dimensions) # Add dimensions to the set.
        except KeyError:
            print(f"Warning: Variable '{variable_name_export}' not found in dataset. Skipping.")
    return list(unique_dimensions) # Convert the set back to a list.

# Print the filtered variable groups and their unique dimensions for confirmation.
i = 0
for variable_group in (variable_group_1, variable_group_2):
    i += 1 
    unique_dimensions_group = get_unique_dimensions(variable_group)
    print(f"{i}. {variable_group}")
    print(f"   Unique dimensions: {unique_dimensions_group}\n")

Available 3D variables in dataset: ['ta', 'u', 'v', 'w', 'wspeed', 'wdir']

1. ['wdir', 'wspeed']
   Unique dimensions: ['y', 'x', 'zu_3d', 'time']

2. ['u', 'v', 'w']
   Unique dimensions: ['x', 'xu', 'zu_3d', 'time', 'zw_3d', 'yv', 'y']



## 5. Data Extraction Logic (2D Surface Slices)
The subset_data function is the core of this notebook's extraction capability. It takes a 3D variable's data for a specific time step and extracts two 2D "surface" slices:
1. The value at the lowest non-zero atmospheric level (often representing ground-adjacent values).
2. The value at the next atmospheric level above the lowest non-zero level.

This is particularly useful for analyzing conditions just above the ground and at a slightly higher elevation.

In [6]:
def subset_data(dataset, variable_name, time_index):
    """
    Extracts 2D "surface" data from a 3D variable for a given time step.
    It returns two subsets:
    - The value at the lowest non-zero atmospheric level.
    - The value at the next atmospheric level above the lowest non-zero level.

    Args:
        dataset (netCDF4.Dataset): The NetCDF dataset containing the variable.
        variable_name (str): The name of the 3D variable to subset (e.g., 'wspeed').
        time_index (int): The specific time step index to extract data for.

    Returns:
        tuple: A tuple containing two NumPy arrays, each of shape (1, 1, y_range, x_range),
               representing the extracted 2D surface slices.
    """
    # Get the variable data object from the dataset.
    var_data = dataset[variable_name]
    
    # Determine the dimensions (levels, y_range, x_range) from the variable's shape,
    # skipping the time dimension (index 0).
    levels, y_range, x_range = var_data.shape[1], var_data.shape[2], var_data.shape[3]

    # Initialize NumPy arrays to store the two 2D subsets.
    # Shape (1, 1, y_range, x_range) maintains compatibility with NetCDF variable dimensions.
    subset_0 = np.zeros((1, 1, y_range, x_range), dtype=np.float32) # For the lowest non-zero level
    subset_1 = np.zeros((1, 1, y_range, x_range), dtype=np.float32) # For the next level

    # Reshape the 3D data for the current time step into a 2D array (levels x total_spatial_points).
    # This allows for efficient vectorized operations along the level axis.
    flat_data = var_data[time_index, :, :, :].reshape(levels, -1)

    # Find the index of the first non-zero level along the vertical (level) axis for each spatial point.
    # `np.argmax(flat_data != 0, axis=0)` returns the first index where the condition is true.
    # If a column is all zeros, argmax returns 0, so additional handling is needed.
    non_zero_indices = np.argmax(flat_data != 0, axis=0)
    
    # Extract values at the `non_zero_indices`.
    non_zero_values = flat_data[non_zero_indices, np.arange(flat_data.shape[1])]

    # Important correction: If a spatial column (all levels at a specific y,x) is entirely zero (e.g., outside domain),
    # `np.argmax` might still return 0, leading to incorrect non-zero values.
    # We set these values to zero where the very first level of data is also zero.
    # This ensures that points with no atmospheric data remain zero.
    non_zero_values[flat_data[0] == 0] = 0

    # Reshape `non_zero_values` back to the original 2D spatial dimensions (y_range, x_range)
    # and assign it to `subset_0`.
    subset_0[0, 0] = non_zero_values.reshape(y_range, x_range)

    # Calculate the indices for the 'next level' (one level above `non_zero_indices`).
    # `np.clip` ensures these indices do not exceed the `levels` boundary.
    next_level_indices = np.clip(non_zero_indices + 1, 0, levels - 1)
    
    # Extract values at `next_level_indices`.
    next_level_values = flat_data[next_level_indices, np.arange(flat_data.shape[1])]
    
    # Apply the same zero-masking for `next_level_values` to handle all-zero columns correctly.
    next_level_values[flat_data[0] == 0] = 0

    # Reshape `next_level_values` and assign it to `subset_1`.
    subset_1[0, 0] = next_level_values.reshape(y_range, x_range)

    return subset_0, subset_1

## 6. Select Dataset and Variable Group for Export
This section defines which simulation dataset (dataset_1 or dataset_2) and which predefined variable group (variable_group_1 or variable_group_2) will be processed for surface data extraction. It also prepares the output filename prefix and suffix based on these selections.

In [7]:
# `ds_index`: Selects which dataset to process.
#   - `1` corresponds to `dataset_1` (Baseline).
#   - `2` corresponds to `dataset_2` (Scenario 1).
ds_index = 1

# `var_group_index`: Selects which variable group to extract.
#   - `1` corresponds to `variable_group_1` (e.g., 'wdir', 'wspeed').
#   - `2` corresponds to `variable_group_2` (e.g., 'u', 'v', 'w').
var_group_index = 1

def get_dataset_and_filename_prefix(dataset_index):
    """
    Retrieves the selected dataset object and constructs a filename prefix
    based on the original NetCDF file's name.

    Args:
        dataset_index (int): Index indicating which dataset to select (1 for dataset_1, 2 for dataset_2).

    Returns:
        tuple: A tuple containing the selected netCDF4 Dataset object and its filename prefix.
    """
    if dataset_index == 1:
        selected_dataset = dataset_1
        # Extract filename (e.g., 'konstanz_4096x4096_v9_Baseline_av_3d_N03.000')
        filename = os.path.basename(file_3d_1)
        # Extract filename prefix (e.g., 'konstanz_4096x4096_v9_Baseline_av_3d_N03.000')
        filename_prefix = filename.split(".nc")[0]
    elif dataset_index == 2:
        selected_dataset = dataset_2
        filename = os.path.basename(file_3d_2)
        filename_prefix = filename.split(".nc")[0]
    else:
        raise ValueError("Invalid dataset_index. Please choose 1 or 2.")
    
    return selected_dataset, filename_prefix

# Assign the chosen variable group and construct a filename suffix for the output.
if var_group_index == 1:
    variable_group = variable_group_1
    filename_suffix = "-wdir-wspeed"
elif var_group_index == 2:
    variable_group = variable_group_2
    filename_suffix = "-u-v-w"
else:
    raise ValueError("Invalid var_group_index. Please choose 1 or 2.")

# Get the selected dataset and its corresponding filename prefix.
dataset, filename_prefix = get_dataset_and_filename_prefix(dataset_index=ds_index)

# Get the unique dimensions for the selected variable group.
unique_dimensions_group = get_unique_dimensions(variable_group)

In [8]:
# Print the title of the selected dataset.
# The title typically contains simulation run information.
print(f"Dataset Title (Part): {dataset.title.split(' ')[4]}") # Example: Extracts a specific part of the title string.

# Print the constructed filename prefix and suffix for the output NetCDF file.
print(f"Output Filename Prefix: {filename_prefix}")
print(f"Output Filename Suffix: {filename_suffix}")

# Print the list of variables included in the selected group.
print(f"Variables selected for export: {variable_group}")

# Print the unique dimensions identified for the selected variable group.
# This confirms the dimensional structure that will be used for the output NetCDF file.
print(f"Unique dimensions for selected variables: {unique_dimensions_group}\n")

Dataset Title (Part): konstanz_4096x4096_v9_Baseline.00
Output Filename Prefix: konstanz_4096x4096_v9_Baseline_av_3d_N03.000
Output Filename Suffix: -wdir-wspeed
Variables selected for export: ['wdir', 'wspeed']
Unique dimensions for selected variables: ['y', 'x', 'zu_3d', 'time']



## 7. Perform Extraction and Save Data
This final and crucial section performs the actual extraction of surface-level 2D data from the 3D variables. It iterates through the selected time steps, extracts the lowest non-zero atmospheric layer and the layer above it for each variable using the subset_data function, and then writes this processed 2D data into a new NetCDF file. This allows for focused analysis on atmospheric conditions near the ground.

In [9]:
# Set the time sequence to process all available time steps.
# This ensures that surface data is extracted for every time point in the simulation.
time_sequence = time_sequence_all

# Define the output directory path.
output_directory = "./output/_02_2D_surface_from_3D/"

# Create the output directory if it does not already exist.
# `os.makedirs` creates all intermediate directories. `exist_ok=True` prevents an error if the directory exists.
os.makedirs(output_directory, exist_ok=True)

# Construct the full output file path.
# Combines the directory, filename prefix, and filename suffix.
output_filepath = os.path.join(output_directory, f"{filename_prefix}{filename_suffix}.nc")

# --- Added check to skip export if file already exists ---
if os.path.exists(output_filepath):
    print(f"Skipping export: File already exists at {output_filepath}")
else:
    # Create a new NetCDF file in write mode ("w").
    # `nc.Dataset` is used for creating and writing to NetCDF files.
    with nc.Dataset(output_filepath, mode="w", format='NETCDF4_CLASSIC') as out_file:
        # --- Copy Global Attributes ---
        # Iterate through all global attributes of the input dataset.
        for attr_name in dataset.ncattrs():
            # Exclude "VAR_LIST" from direct copying as it will be reconstructed.
            if attr_name != "VAR_LIST":
                # Copy each attribute's value to the output file.
                out_file.setncattr(attr_name, dataset.getncattr(attr_name))
        
        # Reconstruct and set the "VAR_LIST" attribute for the output file.
        # This attribute lists the variables contained within the new file, formatted as ';var1;var2;'.
        var_list_str = "".join([f";{var}" for var in variable_group]) + ";"
        out_file.setncattr('VAR_LIST', var_list_str)
        
        # --- Create Dimensions ---
        # Iterate through the unique dimensions required by the `variable_group`.
        for dim_name in unique_dimensions_group:
            # Handling for 'time' dimension:
            if dim_name == 'time':
                # Create 'time' dimension in the output file with the length of the processed time sequence.
                out_file.createDimension(dim_name, len(time_sequence))
                # Create a variable for 'time' and copy its data from the original dataset,
                # but only for the `time_sequence` (if `time_sequence` is a subset of `time_sequence_all`).
                # In this notebook, `time_sequence = time_sequence_all`, so all original times are copied.
                out_variable_dim = out_file.createVariable(dim_name, dataset[dim_name].dtype, (dim_name,))
                out_variable_dim[:] = dataset[dim_name][:]
            
            # Handling for 'z' (vertical) dimension:
            elif dim_name.startswith('z'):
                # For surface extraction, the 'z' dimension is reduced to a size of 1.
                # This represents the chosen surface layer.
                dim_size = 1 
                out_file.createDimension(dim_name, dim_size)
                # Create a variable for 'z' and copy only the first level's value (as we take slices from 3D).
                out_variable_dim = out_file.createVariable(dim_name, dataset[dim_name].dtype, (dim_name,))
                out_variable_dim[:] = dataset[dim_name][:1] # Copy only the first z-level's coordinate.

            # Handling for 'x' and 'y' (horizontal) dimensions:
            elif dim_name.startswith('x') or dim_name.startswith('y'):
                # These dimensions retain their original sizes.
                out_file.createDimension(dim_name, len(dataset.dimensions[dim_name]))
                out_variable_dim = out_file.createVariable(dim_name, dataset[dim_name].dtype, (dim_name,))
                out_variable_dim[:] = dataset[dim_name][:]
                
            # Handling for 'lon' and 'lat' (geographic coordinates) dimensions:
            elif dim_name.startswith('lon') or dim_name.startswith('lat'):
                # These dimensions retain their original sizes.
                out_file.createDimension(dim_name, len(dataset.dimensions[dim_name]))
                out_variable_dim = out_file.createVariable(dim_name, dataset[dim_name].dtype, (dim_name,))
                out_variable_dim[:] = dataset[dim_name][:]
            
        # --- Write Variable Values ---
        # Iterate through each variable name in the selected `variable_group`.
        for variable_name in variable_group:
            # Check if the variable exists in the input dataset.
            if variable_name in dataset.variables:
                # Create the variable in the output NetCDF file.
                # The dimensions are copied from the original variable, but the `z` dimension's size will effectively be 1.
                # Note: `dataset[variable_name].dimensions` will include 'z', which is handled by its dimension size of 1.
                out_variable = out_file.createVariable(variable_name,
                                                       dataset[variable_name].dtype, 
                                                       dataset[variable_name].dimensions, # Use original dimensions for consistency
                                                       fill_value=dataset[variable_name]._FillValue if '_FillValue' in dataset[variable_name].ncattrs() else -9999.0) # Copy fill value

                # Loop through each time index in the `time_sequence`.
                for t_idx, time_step_idx in enumerate(time_sequence):
                    # Use `subset_data` to extract the two 2D surface slices for the current time step.
                    # `data_0` (lowest non-zero layer) and `data_1` (next layer).
                    data_0, data_1 = subset_data(dataset, variable_name, time_step_idx)
                    
                    # Write `data_1` (the next atmospheric layer above the lowest non-zero) to the output file.
                    # `[t_idx, 0, :, :]` specifies writing to the `t_idx`-th time step, 0-th z-layer, and all y, x dimensions.
                    # To save `data_0` instead, change `data_1` to `data_0`.
                    out_variable[t_idx, 0, :, :] = data_1
            else:
                print(f"Warning: Variable '{variable_name}' not found in the dataset. Skipping export for this variable.")

    print(f"Successfully extracted and saved surface data to: {output_filepath}")

Successfully extracted and saved surface data to: ./output/_02_2D_surface_from_3D/konstanz_4096x4096_v9_Baseline_av_3d_N03.000-wdir-wspeed.nc
